In [1]:
import sys
from joblib import load

sys.path.insert(0, '..')
sys.path.insert(0, '../..')
sys.path.insert(0, '../../..')
sys.path.insert(0, '../../../..')
sys.path.insert(0, '../../../../..')
sys.path.insert(0, '../../../../../..')

import torch
from torch import optim

from reimplemented_approaches.proactive_conformance_checking.models import LSTMCollectiveIDP
from reimplemented_approaches.proactive_conformance_checking.training import Training
from reimplemented_approaches.proactive_conformance_checking.data_prep_split_encode import PrefixDataset


In [2]:
# Load encoders:
# Load prepared and encoded datasets
train_set, val_set, test_set = PrefixDataset.load_datasets(save_path="../../../data_preparation/BPIC20/collective/")

print(train_set.tensors[0].size())
print(train_set.tensors[1].size())
print(train_set.tensors[2].size())
print(train_set.tensors[3].size())
print(train_set.tensors[4].size())

encoders = load("../../../data_preparation/BPIC20/collective/encoders.pkl")
print(encoders)

activity_ids = encoders.get('activity_ids') 
activity_ids_vocab_size_with_default = len(list(activity_ids.keys())) + 1 
print("Activities: ", activity_ids_vocab_size_with_default)

resource_ids = encoders.get('resource_ids')
resource_ids_vocab_size_with_default = len(list(resource_ids.keys())) + 1
print("Resources: ", resource_ids_vocab_size_with_default)

month_ids = encoders.get('month_ids') 
month_ids_vocab_size_with_default = len(list(month_ids.keys())) + 1

print("Months: ", month_ids_vocab_size_with_default)

number_trace_attr = train_set.tensors[3].size(1)
print("Number trace attributes: ", number_trace_attr)

number_deviations_y = len([d for d in encoders.get('deviations')])
print("Number deviatons y: ", number_deviations_y)


torch.Size([30100, 24])
torch.Size([30100, 24])
torch.Size([30100, 24])
torch.Size([30100, 1])
torch.Size([30100, 16])
{'activity_ids': {'Declaration APPROVED by ADMINISTRATION': 1, 'Declaration APPROVED by BUDGET OWNER': 2, 'Declaration APPROVED by PRE_APPROVER': 3, 'Declaration FINAL_APPROVED by SUPERVISOR': 4, 'Declaration FOR_APPROVAL by ADMINISTRATION': 5, 'Declaration FOR_APPROVAL by PRE_APPROVER': 6, 'Declaration FOR_APPROVAL by SUPERVISOR': 7, 'Declaration REJECTED by ADMINISTRATION': 8, 'Declaration REJECTED by BUDGET OWNER': 9, 'Declaration REJECTED by EMPLOYEE': 10, 'Declaration REJECTED by MISSING': 11, 'Declaration REJECTED by PRE_APPROVER': 12, 'Declaration REJECTED by SUPERVISOR': 13, 'Declaration SAVED by EMPLOYEE': 14, 'Declaration SUBMITTED by EMPLOYEE': 15, 'Payment Handled': 16, 'Request Payment': 17}, 'resource_ids': {'STAFF MEMBER': 1, 'SYSTEM': 2}, 'month_ids': {'10_2017': 1, '10_2018': 2, '11_2017': 3, '11_2018': 4, '12_2017': 5, '12_2018': 6, '1_2017': 7, '1_20

In [3]:
# device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# device = torch.device("cpu")
print("Using device:", device)

Using device: cuda


In [4]:
# Embedding
embedding_dim = 16
# lstm hidden size
lstm_hidden = 128
# fully connected hidden
fc_hidden = 128
# dropout probability
p_dropout = 0.1

model = LSTMCollectiveIDP(activity_vocab_size=activity_ids_vocab_size_with_default,
                          resource_vocab_size=resource_ids_vocab_size_with_default,
                          month_vocab_size=month_ids_vocab_size_with_default,
                          num_trace_features=number_trace_attr,
                          num_output_labels=number_deviations_y,
                          # hyperparams provided in paper:
                          embedding_dim=embedding_dim,
                          lstm_hidden=lstm_hidden,
                          fc_hidden=fc_hidden,
                          dropout=p_dropout,
                          device=device)

In [5]:
# not mentioned in the paper:
# from code in paper
batch_size=128
# from code in paper
shuffle = True
epochs = 300 # 300 if early stopping (20% val from all train)
# from code in paper
learning_rate = 0.0001

optimizer = optim.Adam(model.parameters(), lr=learning_rate)

optimizer_values = {"optimizer":optimizer,
                    "epochs":epochs,
                    "mini_batches":batch_size,
                    "shuffle": shuffle}

# Training with ealy stopping according to journal paper: patience:10, min_delta = 0
training = Training(model=model,
                    train_set=train_set,
                    val_set=val_set,
                    optimizer_values=optimizer_values,
                    loss_mode = 'collective',
                    device=device,
                    saving_path='./LSTM_collecctive_IDP.pkl')

history = training.train()

Epoch 1/300:   0%|          | 0/236 [00:00<?, ?it/s]

Epoch 1/300 — train loss: 0.1776, val loss: 0.0533


Epoch 2/300:   0%|          | 0/236 [00:00<?, ?it/s]

Epoch 2/300 — train loss: 0.0524, val loss: 0.0479


Epoch 3/300:   0%|          | 0/236 [00:00<?, ?it/s]

...

Epoch 109/300 — train loss: 0.0150, val loss: 0.0137


Epoch 110/300:   0%|          | 0/236 [00:00<?, ?it/s]

Epoch 110/300 — train loss: 0.0150, val loss: 0.0136


Epoch 111/300:   0%|          | 0/236 [00:00<?, ?it/s]

Epoch 111/300 — train loss: 0.0150, val loss: 0.0137
Early stopping triggered.
